In [ ]:
from citadel import Citadel
import json
import pandas as pd
from copy import deepcopy
import pdb
import arrow
import re
from plpygis import Geometry

In [ ]:
with open('config/citadel_config.json', 'r') as fp:
    citadel_config = json.load(fp)
citadel = Citadel(citadel_config['hostname'], citadel_config['apikey'])

In [ ]:
df = pd.read_csv('data/sample_waze.csv')
df.head()

In [ ]:
metadata_columns = ['street', 'city', 'road_type']
data_columns = ['length', 'delay', 'speed', 'level']

In [ ]:
unit_map = {
    'traffic_length': 'MI',
    'traffic_delay': 'SEC',
    'traffic_level': 'no_unit',
    'traffic_speed': 'MI-PER-HR',
}
point_type_map = {
    'length': 'traffic_length',
    'delay': 'traffic_delay',
    'level': 'traffic_level',
    'speed': 'traffic_speed'
}

def get_name(srcid, point_type):
    return 'waze_{0}_{1}'.format(srcid, point_type)

def geom2xys(geom):
    return Geometry(geom).geojson['coordinates']

def str2xys(linestr):
    coo_strs = re.findall('\(.+\)', linestr)[0][1:-1].split(',')
    coordinates = []
    for coo_str in coo_strs:
        coo_strs = coo_str.split()
        lng = float(coo_strs[0])
        lat = float(coo_strs[1])
        coordinates.append([lng, lat])
    return coordinates

In [ ]:
srcids = df['uuid'].unique()
print(len(srcids))

In [ ]:
# Structuring Metadata 
metadatas = dict()
cnt = 0
for srcid in srcids:
    if cnt % 10000 == 0:
        print('created {0} points'.format(cnt))
    traffic = df.loc[df['uuid']==srcid].iloc[0]
    traffic = dict((k, traffic[k]) for k in metadata_columns)
    #for srcid in srcids:
    traffic = dict()
    traffic['waze_srcid'] = str(srcid)
    for col in data_columns:
        point_type = 'traffic_' + col
        point = deepcopy(traffic)
        name = get_name(srcid, point_type)
        point['pointType'] = point_type
        point['unit'] = unit_map[point_type]
        point['name'] = name
        metadatas[name] = point
        uuids = citadel.query_points({'name': name})
        if not uuids:
            uuid = citadel.create_point(point)
            if not uuid:
                pdb.set_trace()
        else:
            uuid = uuids[0]
        pdb.set_trace()
        point['uuid'] = uuid
        metadatas[name] = point

In [ ]:
with open('metadata/waze-metadata', 'w') as fp:
    json.dump(metadatas, fp)

In [ ]:
data = []
cnt = 0
for _, row in df.iterrows():
    if row['uuid'] not in srcids:
        continue
    cnt += 1
    if cnt % 100000 == 0:
        print('pushed {0} data'.format(cnt))
    srcid = row['uuid']
    t = arrow.get(row['waze_timestamp']).timestamp * 1000
    line = geom2xys(row['geom'])
    for data_col in data_columns:
        point_type = point_type_map[data_col]
        name = get_name(srcid, point_type)
        uuid = metadatas[name]['uuid']
        datum = {
            'value': row[data_col],
            'coordinates': line,
            'timestamp': t,
            'geometryType': 'line',
            'uuid': uuid,
        }
        data.append(datum)
    if len(data) > 1000:
        res = citadel.post_data(data)
        if not res:
            print("Failed")
        data = []

In [ ]:
uuid = list(metadatas.values())[0]['uuid']

In [ ]:
begin_time = arrow.get()
citadel.get_data(uuid=uuid)
end_time = arrow.get()
print(end_time - begin_time)

In [ ]:
begin_time = arrow.get()
data= citadel.query_bbox(-117.006479, 32.621095, -117.002479, 32.651195, 1486063580000, 1486203580000, [uuid])
end_time = arrow.get()
print(end_time - begin_time)